# Tutorial 6: Multiprocessing

In [1]:
import time
import random
import yaml
import os
from pprint import pprint

from theflow import Function, Param, Node, load
from theflow.utils.multiprocess import parallel

class Sum1(Function):
    a: int

    def run(self, x, y, w, task_number) -> int:
        time.sleep(w)
        print(os.getpid(), self._ff_run_id, self._ff_prefix, self._ff_name, task_number)
        return self.a + x + y

class Plus(Function):
    a: int
    func: Function = Node(default=Sum1.withx(a=1))

    def run(self, x, n_times, n_processes) -> int:
        print(self._ff_childs_called)
        tasks = [{"x": x, "y": self.a, "w": random.random() * 5, "task_number":_} for _ in range(n_times)]
        results = list(parallel(self, "func", tasks, processes=n_processes))
        return sum(results)

In [2]:
pip = Plus(a=1, func=Sum1(a=1))
output = pip(1, 10, 2)

{}
49631 17003595918709896 . func 0
49634 17003595918709896 . func[1] 1
49631 17003595918709896 . func[2] 2
49631 17003595918709896 . func[4] 4
49634 17003595918709896 . func[3] 3
49631 17003595918709896 . func[5] 5
49634 17003595918709896 . func[6] 6
49631 17003595918709896 . func[7] 7
49634 17003595918709896 . func[8] 8
49631 17003595918709896 . func[9] 9


In [3]:
pip.context.get_all_contexts()

{'__main__.Plus|17003595918709896': {'run_id': '17003595918709896'},
 '__main__.Plus|17003595918709896|.': {},
 '__main__.Plus|17003595918709896|__progress__': {'name': '__main__.Plus',
  'id': '17003595918709896',
  '.': {'status': 'run',
   'input': {'args': (1, 10, 2), 'kwargs': {}},
   'output': 30},
  '.func': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 3.24800759033075, 'task_number': 0}},
   'output': 3},
  '.func[1]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 3.6689934325906943, 'task_number': 1}},
   'output': 3},
  '.func[2]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 0.6497443397620883, 'task_number': 2}},
   'output': 3},
  '.func[3]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 1.8750110944821379, 'task_number': 3}},
   'output': 3},
  '.func[4]': {'status': 'run',
   'input': {'args': (),
    'kwargs': {'x': 1, 'y': 1, 'w': 1.3563

-------